# Lab session 2: Probablistic models; generative modeling for anomaly detection

## Task specification
In this lab session, you will follow the demo code provided below, and complete the following two tasks:
* Compare and analyze the performance of several popular **probabilistic models** on a number of simulated classification tasks. 
* Understand why **generative modeling** is useful for detect anomalies.
* Apply the model of your choice to a **anomaly detection** task, and report the performance of your model.

## Learning objectives
By the end of this lab, you should understand the following:
* How to *apply generative models* in python such as Gaussian Naive Bayes for anomaly detection tasks?
* What are the *design choices* of probablistic models? When should we care about descriminative models? When should we use generative models? 
* What assumptions are made for different generative models, such as Gaussian Naive Bayes and Fisher's LDA?
* How to *evaluate and visualize* the outcome of probabilistic models?

In [ ]:
# Python Notebook Commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# General math and plotting modules.
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import linalg

# Widget and formatting modules
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed
from matplotlib import rcParams
import matplotlib as mpl 
from scipy.stats import multivariate_normal

# If in your browser the figures are not nicely vizualized, change the following line. 
rcParams['figure.figsize'] = (10, 5)
rcParams['font.size'] = 16

# Machine Learning library. 
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import datasets

import warnings
warnings.filterwarnings("ignore")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Problem setup: synthetic dataset

Similar to the previous lab session, before moving on to our case study, we would like to create specific training datasets to evaluate different algorithms under a controled environment. We now create a few synthetic datasets that capture such data distributions.


In [ ]:
def get_dataset(dataset, n_samples=200, noise=0.3):
    if dataset == 'linear':
        X, Y = linear_separable_data(n_samples, noise=noise, dim=2) 
        Y = (Y + 1) // 2
    elif dataset == '2-blobs':
        X, Y = datasets.make_classification(n_classes=2, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '3-blobs':
        X, Y = datasets.make_classification(n_classes=3, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '4-blobs':
        X, Y = datasets.make_classification(n_classes=4, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8) 
    elif dataset == 'high-dim':
        X, Y = datasets.make_classification(n_classes=3, n_features=10, n_informative=6, n_samples=n_samples,
                                            random_state=8)
        
    elif dataset == 'circles':
        X, Y = datasets.make_circles(noise=noise, n_samples=n_samples, factor=.5)
    elif dataset == 'moons':
        X, Y = datasets.make_moons(noise=noise, n_samples=n_samples)
    elif dataset == 'imbalanced':
        X, Y = linear_separable_data(n_samples, noise=noise, dim=2, num_negative=int(n_samples * 0.2))
        Y = (Y + 1) // 2
    elif dataset == 'correlated':
        X, Y = datasets.make_classification(n_classes=2, n_features=2, n_informative=1, n_redundant=1,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)  
    elif dataset == 'iris':
        X, Y = datasets.load_iris(return_X_y=True)
    elif dataset == 'mnist':
        X, Y = datasets.load_digits(return_X_y=True)
    elif dataset == 'wine':
        X, Y = datasets.load_wine(return_X_y=True)
    return X, Y


def linear_separable_data(num_positive, num_negative=None, noise=0., offset=1, dim=2):
    if num_negative is None:
        num_negative = num_positive

    x = offset + noise * np.random.randn(num_positive, dim)
    y = 1 * np.ones((num_positive,), dtype=np.int)

    x = np.concatenate((x, noise * np.random.randn(num_negative, dim)), axis=0)
    y = np.concatenate((y, -1 * np.ones((num_negative,), dtype=np.int)), axis=0)

    x = np.concatenate((x, np.ones((num_positive + num_negative, 1))), axis=1)

    return x, y


## Preliminaries: helper functions for data analytics and visualization

We also need several helper functions for visualizing the data distribution and qualitatively assessing the model behavior.

In [ ]:
def plot_ellipse(mean, covar, color, ax):
    v, w = linalg.eigh(covar)
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    u = w[0] / linalg.norm(w[0])

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan(u[1] / u[0])
    angle = 180. * angle / np.pi  # convert to degrees


    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle,  color=color)
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.5)
    ax.add_artist(ell)


def plot_model(model, X, Y, means, covariances):
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    pmap = plt.cm.cividis_r
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)
        
    # PREDICT
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]
    C = model.predict(xy)
    P = model.predict_proba(xy)
    H = -(P * model.predict_log_proba(xy)).sum(axis=1)
    
    P = P.max(axis=1)
    # Put the result into a color plot
    C = C.reshape(xx.shape)
    P = P.reshape(xx.shape)
    H = H.reshape(xx.shape)
    
    # PLOTS
    fig, axes = plt.subplots(1, 3)

    for ax in axes:
        ax.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.jet)
        
        if means is not None:
            for i in range(num_classes):
                plot_ellipse(means[i], covariances[i], cmap(norm(i)), ax)

        ax.set_xlim(xx.min()+h, xx.max()-h)
        ax.set_ylim(yy.min()+h, yy.max()-h)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_aspect('equal')

    axes[0].set_title('Classification Boundary')
    axes[0].contourf(xx, yy, C, cmap=cmap, alpha=0.5)
    
    axes[2].set_title('Prediction Probabilities')
    cf = axes[2].contourf(xx, yy, P, cmap=pmap, alpha=0.5, vmin=1. / num_classes, vmax=1)
    
    axes[1].set_title('Probabilistic Boundary')
    axes[1].contourf(xx, yy, P * C, cmap=cmap, alpha=0.5)
    plt.show()
    # Plot also the training point
    

## Discriminative models vs Generative models 

In module 1, we discussed both discriminative and generative classification models, with the key differences in the modeling objective:

* **discriminative models** aim to estimate conditional distribution $P(Y | X)$
* **generative models** aim to estimate joint distribution $P(Y, X)$

As the name suggests, a generative model not only focuses on making classification predictions for any given data points, but also describes how data is generated, in terms of a probabilistic model. Since one can derive conditional distribution from the joint distribution, we conclude that generative models are **strictly more powerful** than descriminative models, but at what cost?

In the following, we will dive into a few concrete generative classification models, starting from ones that make strong distributional assumptions of the data, namely the Naive Bayes model, and then look into techniques that lift those assumptions for more general generative classification tasks.



### Gaussian Naive Bayes (GNB)

As introduced in Module 1 video, GNB models class label as generated from categorical variable,
 $$P(Y=y) = p_y, y\in \mathcal{Y} = \{1, \dots, c\}$$
and models features as conditionally independent Gaussians
$$P(X_{[1]},\dots, X_{[d]} | Y) = \prod_{i=1}^d P(X_{[i]} | Y)$$
$$P(x_{[i]} | y) = \mathcal{N}\left(x_{[i]} | \mu_{y,[i]}, \sigma^2_{y,[i]}\right)$$

Given class label, each feature is generated independently of the other features. Learning therefore reduces to specifying the label distribution $P(Y)$ and feature distribution $P(X_{[i]} | Y)$.





### Fisher's linear discriminant analysis (Fisher's LDA)

A more general generative model is to lift the ``Naive'' assumption made by Gaussian Naive Bayes, i.e. the conditional independence assumption. 

Suppose we fix $c=2$, $p_+=p_-=0.5$, and assume feature covariances among two classes are equal: $\Sigma_+ = \Sigma_- = \Sigma$}. Then this generative model is called *Fisher's linear discriminant analysis*. 

The function $f(x) = \log \frac{P(Y=+1 \mid x)}{P(Y=-1 \mid x)}$  is called *discriminant function*. Under the above modeling assumptions, Fisher's LDA predicts
\begin{align*}
y=\text{sign}(f(x)) = \text{sign}(w^\top x + b)
\end{align*}
where $w=\hat{\Sigma}^{-1} \left(\hat{\mu}_{+}- \hat{\mu}_{-}\right)$, and $b=\frac12\left( 
\hat{\mu}_{-}^{\top}
\hat{\Sigma}^{-1} 
\hat{\mu}_{-} - 
\hat{\mu}_{+}^{\top}
\hat{\Sigma}^{-1} 
\hat{\mu}_{+}
\right).$

Fisher's LDA uses the discriminant function
\begin{align*}
& f(x) = \log \frac{P(Y=+1 \mid x)}{P(Y=-1 \mid x)} := w^\top x + b\\
\Leftrightarrow~ &
P(Y=+1 \mid x) = \frac{1}{1+\exp{(-f(x))}} = \sigma(f(x))
\end{align*}

Therefore, the class probability of LDA is
\begin{align*}
P(Y=+1 \mid x) = \sigma(w^\top x + b)
\end{align*}
This is of the same form as logistic regression. 




## Comparison of the decision boundaries between generative and discriminative classification models

In this task, you will examine how the modeling assumptions shape the decision boundary of each probabilistic model, in the context of classification.

As you go through the following demo, try to answer the following questions:

* When does GNB and Logistic Regressian make the same prediction?
* Does one model behavior strictly better than another? If yes, under what cost?

In [ ]:
rcParams['figure.figsize'] = (20, 15)
rcParams['font.size'] = 16

# %% FLOAT WIDGETS
noise_widget = ipywidgets.FloatSlider(
    value=0.2, min=0, max=1, step=0.01, description='Noise:', continuous_update=False)

def generative_modelling(dataset, model, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    num_classes = len(np.unique(Y))
    X = X[:, :2]
    if model == 'Naive Bayes':
        model = GaussianNB().fit(X, Y)
        mean, covariance = model.theta_, [np.diag(model.sigma_[i]) for i in range(len(model.classes_))]
        priors = model.class_prior_
    elif model == 'FisherLDA':
        model = LDA(store_covariance=True, priors=np.ones(num_classes) / num_classes).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_
    elif model == 'LogisticRegression':
        model = LogisticRegression().fit(X, Y)
        mean, covariance = None, None
        priors = None
        
    plot_model(model, X, Y, mean, covariance)
    if priors is not None:
        print(f"Class Priors: {priors}")
    
interact(generative_modelling, 
         dataset=['2-blobs', 'linear',  'correlated',  'imbalanced', '3-blobs', '4-blobs', 'circles', 'moons', 'wine'], 
         model=['Naive Bayes', 'FisherLDA', 'LogisticRegression'], noise=noise_widget);
        

interactive(children=(Dropdown(description='dataset', options=('2-blobs', 'linear', 'correlated', 'imbalanced'…

## LDA for dimensionality reduction

Play with the code below by switching the datasets, and summarize your understanding of the two dimensionality reduction algorithms.

You should discuss with your group members, to see if you can reach the following consensus:

*   LDA attempts to find a 1-dim subspace that **maximizes class separability**, i.e. ratio of between and within class variances.
*   PCA (1 component) **maximizes the variance** of the resulting 1-dim projection. 


In [ ]:
rcParams['figure.figsize'] = (20, 5)
rcParams['font.size'] = 16

dataset, noise = 'high-dim', 0.3
dataset = 'iris'

def dim_reduction(dataset, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)

    # X = X[:, :2]

    pca = PCA(n_components=2).fit(X)
    Xpca = pca.transform(X)
    
    if num_classes == 2:
        lda = LDA(n_components=2, store_covariance=True, solver='eigen').fit(X, Y)
        Xlda = np.dot(X, lda.scalings_)[:, :2]
        means = np.dot(lda.means_, lda.scalings_)[:, :2]
    else:
        lda = LDA(n_components=2, store_covariance=True).fit(X, Y)
        Xlda = np.dot(X - lda.xbar_, lda.scalings_)[:, :2]

        means = np.dot(lda.means_ - lda.xbar_, lda.scalings_)[:, :2]

    cov = lda.scalings_.T @ lda.covariance_ @ lda.scalings_ 
    cov = cov[:2, :2]

    fig, axes = plt.subplots(1, 3)

    axes[0].scatter(X[:, 0], X[:, 1], c=Y, cmap=cmap)
    axes[0].set_title('First 2 Dimensions')

    axes[1].scatter(Xpca[:, 0], Xpca[:, 1], c=Y, cmap=cmap)
    axes[1].set_title(f"PCA Explained Var {np.array2string(pca.explained_variance_ratio_, precision=2, separator=',')}")

    axes[2].scatter(Xlda[:, 0], Xlda[:, 1], c=Y, cmap=cmap)
    # axes[0].set_aspect('equal')
    for i in range(num_classes):
        plot_ellipse(means[i], cov, cmap(norm(i)), axes[2])
    axes[2].set_title(f"LDA Explained Var {np.array2string(lda.explained_variance_ratio_, precision=2, separator=',')}")
    
interact(
    dim_reduction, 
    dataset=['high-dim', 'iris',  'mnist', 'wine'], 
    noise=noise_widget
);


interactive(children=(Dropdown(description='dataset', options=('high-dim', 'iris', 'mnist', 'wine'), value='hi…

## Generative modeling for Anomally Detection

So far, you have seen the similarity and difference between generative and discriminative models for **classification** tasks.

In the following, we look into an anomaly detection task, where discriminative models falls short. Work with your teammates, and try to answer the following questions:

* Why can generative models be used to detect outliers/anomalies?
* Can you tell if Fisher's LDA is robust against violation of its modeling assumption (i.e. equal class probability and intra-class covariance)?
* Can you use logistic regression to detect outliers?
* Can you think of any application scenarios in the cybersecurity domain, where generative models could be a particularly good fit?


In [ ]:
def get_anomalies(X, mean, covariance, priors, threshold):
    num_classes = len(covariance)
    P = np.zeros(X.shape[0])
    m = 0
    for i in range(num_classes):
        d = multivariate_normal(mean[i], covariance[i])
        d.logpdf(X)
        P += priors[i] * 2 * d.pdf(X)
        m += priors[i] * d.pdf(mean[i])
    return P < threshold

In [ ]:
rcParams['figure.figsize'] = (20, 10)
rcParams['font.size'] = 16

def anomaly_detection(dataset, threshold,  model_name, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    X = X[:, :2]

    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)

    if model_name == 'Naive Bayes':
        model = GaussianNB().fit(X, Y)
        mean, covariance = model.theta_, [np.diag(model.sigma_[i]) for i in range(len(model.classes_))]
        priors = model.class_prior_
    elif model_name == 'LDA':
        model = LDA(store_covariance=True).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_

    fig, axes = plt.subplots(2, 2)
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]


    for j in range(2):
        axes[j, 0].scatter(X[:, 0], X[:, 1], c=Y, marker='o', cmap=cmap)
        for i in range(num_classes):
            plot_ellipse(mean[i], covariance[i], cmap(norm(i)), axes[j, 0]);

    axes[0, 0].set_title('Original Data')
    C = model.predict(xy)

    axes[1, 0].set_title('Original Classification Boundary')
    axes[1, 0].contourf(xx, yy, C.reshape(xx.shape), cmap=cmap, alpha=0.5);


    idx = get_anomalies(X, mean, covariance, priors, threshold)
    for j in range(2):
        axes[j, 1].scatter(X[idx, 0], X[idx, 1], c=Y[idx], marker='x', cmap=cmap, label='Anomalies')
        axes[j, 1].scatter(X[~idx, 0], X[~idx, 1], c=Y[~idx], marker='o', cmap=cmap, label='Non-anomalies')

    axes[0, 1].set_title('Anomaly Detection')
    axes[0, 1].legend()

    idxy = get_anomalies(xy, mean, covariance, priors, threshold)
    C = 2 * C
    C[idxy] = 1
    axes[1, 1].set_title('Anomalous Classification Boundary')
    axes[1, 1].contourf(xx, yy, C.reshape(xx.shape), cmap=cmap, alpha=0.5);
    
    for row in axes:
        for ax in row:
            ax.set_xlim(xx.min() + h, xx.max() - h)
            ax.set_ylim(yy.min() + h, yy.max() - h)
    
interact(anomaly_detection, 
         dataset=['linear', '2-blobs', 'correlated',  'imbalanced', '3-blobs', '4-blobs', 'circles', 'moons', 'iris', 'mnist'], 
         threshold=ipywidgets.FloatSlider(
             value=.1, min=0, max=.5, step=.01, description='Threshold', continuous_update=False),
         model_name=['Naive Bayes', 'LDA'], 
         noise=ipywidgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='Noise:', continuous_update=False));


interactive(children=(Dropdown(description='dataset', options=('linear', '2-blobs', 'correlated', 'imbalanced'…

## Exercise: Network attack detection

In this exercise, try out the above generative models on the following network anomaly detection task. For your convenience, we have provided the data loader (see below). Try to plug in Naive Bayes and/or LDA to this dataset.

We will use the [UCI detection_of_IoT_botnet_attacks_N_BaIoT Data Set](https://archive.ics.uci.edu/ml/datasets/detection_of_IoT_botnet_attacks_N_BaIoT) in this exercise. The dataset contains *real* benign and Malicious traffic data, gathered from 9 commercial IoT devices authentically infected by Mirai and BASHLITE. For demonstration purpose, we provide a snippet of the full dataset, accessible from the link below:

* [benign_traffic.csv](https://yuxinchen.org/files/teaching/ml4security/module1/data/anomalydet/benign_traffic.csv)
* [scan.csv](https://yuxinchen.org/files/teaching/ml4security/module1/data/anomalydet/scan.csv)

The dataset creators (Yair Meidan, Michael Bohadana, Yael Mathov, Yisroel Mirsky, Dominik Breitenbacher, Asaf Shabtai and Yuval Elovici) provided a detailed description of the dataset [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00442/N_BaIoT_dataset_description_v1.txt). 

Complete the following tasks:
* Justify your choice of anomaly detection models
* Report the detection precisian and recall on the test data.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

def load_data(Train=False):

    url_benign = 'https://yuxinchen.org/files/teaching/ml4security/module1/data/anomalydet/benign_traffic.csv'
    url_scan = 'https://yuxinchen.org/files/teaching/ml4security/module1/data/anomalydet/scan.csv'

    cdata=pd.read_csv(url_benign)
    scan=pd.read_csv(url_scan)

    print(cdata)
    print(scan)

    #checking the dimensions
    print("Cdata SHape: ",cdata.shape)
    #print(cdata.head())
    print("Scan Shape : ",scan.shape)

    #Tagging Clean Traffic as 0 and Threat Traffic as 1
    scan['Out']=1
    cdata['Out']=0

    #combining two DataFrames
    comb=pd.concat([cdata,scan],axis=0)
    print("The COMBINED shape : ",comb.shape)
    comb=shuffle(comb)
    Output=comb['Out']
    comb=comb.drop(['Out'],axis=1)
    comb=comb.drop(['HpHp_L0.01_pcc'],axis=1)
    comb1=comb.iloc[:,:28] # Removing all the Outlier Columns

    Output=np.array(Output).flatten()
    # print("After remove: ",comb.shape)
    # print("\nThe OUTPUt is : \n",Output)
    # print("\nOutput SHAPE : ",Output.shape)

    #Standardization
    comb_std=(comb-(comb.mean()))/(comb.std())
    comb_std_arra=np.array(comb_std)

    #Using SKLearn
    scale=StandardScaler()
    scale.fit(comb1)
    scale.transform(comb1)

    #Normalization
    comb_norm=(comb-(comb.min()))/((comb.max())-comb.min())

    if Train:
        # returns X_train, X_test, y_train, y_test
        return train_test_split(comb1, Output, test_size=0.2, random_state=RandomState())
    else:
        return comb1, Output

load_data()


      MI_dir_L5_weight  MI_dir_L5_mean  ...  HpHp_L0.01_covariance  HpHp_L0.01_pcc
0             1.000000       60.000000  ...           0.000000e+00    0.000000e+00
1             1.000000       60.000000  ...           0.000000e+00    0.000000e+00
2             1.000000       60.000000  ...           0.000000e+00    0.000000e+00
3             1.000000      590.000000  ...           0.000000e+00    0.000000e+00
4             1.927179      590.000000  ...           0.000000e+00    0.000000e+00
...                ...             ...  ...                    ...             ...
9994          1.000000      330.000000  ...           4.240000e-29    0.000000e+00
9995          1.998594      330.000000  ...          -1.110000e-28   -3.820000e-18
9996          1.000000       60.000016  ...           1.240000e-28    1.110000e-16
9997          1.000000      330.000000  ...           2.530000e-29    1.740000e-18
9998          1.999917      330.000000  ...          -6.640000e-29   -4.560000e-18

[99

(      MI_dir_L5_weight  MI_dir_L5_mean  ...  H_L0.1_variance  H_L0.01_weight
 8493         91.207756       60.000000  ...     3.542620e-03     8488.019209
 2855          1.000000      330.000000  ...     1.051926e+04       17.332664
 6372         52.996729       60.000000  ...     1.081440e-02     7189.487869
 4725         13.555601       60.000000  ...     9.090000e-13     6101.929069
 6678         36.942360       60.000000  ...     9.206376e-03     7323.831671
 ...                ...             ...  ...              ...             ...
 9502        105.642792       60.246913  ...     1.508182e+01      137.917574
 2063         66.011095       60.000000  ...     1.360000e-12     4181.485907
 7277          1.999934      330.000000  ...     8.523876e+03       19.020455
 4516         95.317070       60.000000  ...     3.640000e-12     5987.084007
 1099         63.916873       60.000000  ...     7.730000e-12     3407.782062
 
 [19998 rows x 28 columns], array([1, 0, 1, ..., 0, 1, 1]))